<div style="display: flex; align-items: flex-start;">
  <div>
      <h1>CS 195: Natural Language Processing</h1>
      <h2>Neural Language Modeling</h2>
      </br>
    <a href="https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_1_NeuralLanguageModeling.ipynb">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
  </div>
  <div style="margin-left: 20px;">
    <img src="images/dalle_neural_net_viz.png" width="500" style="display: block;">
  </div>
</div>


**Cover Illustration:** generated by Dall E using the ChatGPT 4 interface, prompted for a visualization of the network used in the code below. *That's not what I meant.*

## Announcement

AI - English Faculty Candidate: Gabriel Ford

Meeting with students: Thursday at 3:30pm in Howard 309

Scholarly Presentation: Friday at 9:00am in Howard ???

## Reference

SLP: Neural Networks and Neural Language Models, Chapter 7 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/7.pdf



In [1]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [2]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])


Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Review: Text Classification with Keras

Last time, we saw 
* how to do text classification when there are more than 2 classes
    - one hot encoded output layer, one node per class, *softmax* output
    - categorical crossentropy loss
* embedding layer
    - pad sequences to all be same length
    - learn vector for each word representing word semantics
    
<div>
    <img src="images/neural_text_classification.png">
</div>

image source: SLP Fig. 7.11, https://web.stanford.edu/~jurafsky/slp3/7.pdf

*pooling* combines/aggregates all of the embeddings in some way

In [3]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, GlobalMaxPooling1D, GlobalAveragePooling1D
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
#use one of these instead of Flatten() to try a pooling method
#model.add(GlobalMaxPooling1D())
#model.add(GlobalAveragePooling1D())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
3750/3750 [==============================] - 39s 10ms/step - loss: 0.3229 - accuracy: 0.8802
Epoch 2/10
3750/3750 [==============================] - 41s 11ms/step - loss: 0.1127 - accuracy: 0.9628
Epoch 3/10
3750/3750 [==============================] - 39s 10ms/step - loss: 0.0313 - accuracy: 0.9912
Epoch 4/10
3750/3750 [==============================] - 37s 10ms/step - loss: 0.0166 - accuracy: 0.9961
Epoch 5/10
3750/3750 [==============================] - 36s 10ms/step - loss: 0.0107 - accuracy: 0.9972
Epoch 6/10
3750/3750 [==============================] - 36s 10ms/step - loss: 0.0074 - accuracy: 0.9977
Epoch 7/10
3750/3750 [==============================] - 36s 10ms/step - loss: 0.0056 - accuracy: 0.9982
Epoch 8/10
3750/3750 [==============================] - 37s 10ms/step - loss: 0.0044 - accuracy: 0.9984
Epoch 9/10
3750/3750 [==============================] - 36s 10ms/step - loss: 0.0036 - accuracy: 0.9985
Epoch 10/10
238/238 [==============================] - 0s 470us/

## Neural Language Modeling

**Neural Language Modeling:** predict next word(s) from previous ones - like what we did with Markov models

Like classification, but output is softmax of every possible word in the vocabulary

Often a first step before extending the model to do summarization, translation, dialog, and other NLP tasks

<div>
    <img src="images/neural_language_modeling.png">
</div>

image source: SLP Fig. 7.13, https://web.stanford.edu/~jurafsky/slp3/7.pdf

## A Neural Language Model in Keras

Let's start by sampling some data from our news dataset

Then split into a training and testing set

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np
import random

data = load_dataset("ag_news")

data_subset, _ = train_test_split(data["train"]["text"],train_size=1000)
train_data, test_data = train_test_split(data_subset,train_size=0.8)

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_subset)
vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary size:",vocabulary_size)

# Convert text to sequences of integers
train_texts = tokenizer.texts_to_sequences(train_data)




Vocabulary size: 7867


## Preparing training examples

We want to take the sequences like 

In [10]:
print( train_texts[0] )

[1004, 200, 749, 50, 4145, 749, 50, 1197, 1468, 24, 2574, 4, 1004, 4, 3, 153, 2, 642, 216, 1861, 2, 4146, 182, 9, 10, 1862, 374, 8, 266, 1469]


and slide a window across to predict the next word

In [11]:
print("Use",train_texts[0][0:5],"to predict",train_texts[0][5])
print("Use",train_texts[0][1:6],"to predict",train_texts[0][6])
print("Use",train_texts[0][2:7],"to predict",train_texts[0][7])
print("Use",train_texts[0][3:8],"to predict",train_texts[0][8])
print("etc.")

Use [1004, 200, 749, 50, 4145] to predict 749
Use [200, 749, 50, 4145, 749] to predict 50
Use [749, 50, 4145, 749, 50] to predict 1197
Use [50, 4145, 749, 50, 1197] to predict 1468
etc.


## Group Discussion

What data structures (lists, matrixes, etc.) do we need to prepare to make this a classification problem?

## Preparing all of the examples

In [12]:
# Decide the sequence length
sequence_length = 5  # Length of the input sequence before predicting the next word

# Create the sequences
predictor_sequences = []
targets = []
for text in train_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)

        
print("Number of sequences:",len(predictor_sequences))


print("First train text:",train_texts[0])
print("Example sequence 0:",predictor_sequences[0]," target:",targets[0])
print("Example sequence 1:",predictor_sequences[1]," target:",targets[1])
print("Example sequence 2:",predictor_sequences[2]," target:",targets[2])
print("Example sequence 3:",predictor_sequences[3]," target:",targets[3])
print("Example sequence 4:",predictor_sequences[4]," target:",targets[4])
print("Example sequence 5:",predictor_sequences[5]," target:",targets[5])


Number of sequences: 27669
First train text: [1004, 200, 749, 50, 4145, 749, 50, 1197, 1468, 24, 2574, 4, 1004, 4, 3, 153, 2, 642, 216, 1861, 2, 4146, 182, 9, 10, 1862, 374, 8, 266, 1469]
Example sequence 0: [1004, 200, 749, 50, 4145]  target: 749
Example sequence 1: [200, 749, 50, 4145, 749]  target: 50
Example sequence 2: [749, 50, 4145, 749, 50]  target: 1197
Example sequence 3: [50, 4145, 749, 50, 1197]  target: 1468
Example sequence 4: [4145, 749, 50, 1197, 1468]  target: 24
Example sequence 5: [749, 50, 1197, 1468, 24]  target: 2574


## Padding

Some of the sequences might be really short - so we'll pad them just in case.

In [13]:
# Pad sequences to ensure uniform length
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

## The target output

Since we're making this into a classification problem, the output layer needs to have one node for each word in the vocabulary. 

Target values need to be transformed into a one-hot encoded vector

In [16]:

# Convert output to one-hot encoding
target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

print("Predictors words 0:", predictor_sequences_padded[0])
print("target word 0:", targets[0])
print("target word 0 one hot encoded:", target_word_one_hot[0])

Predictors words 0: [1004  200  749   50 4145]
target word 0: 749
target word 0 one hot encoded: [0. 0. 0. ... 0. 0. 0.]


## Preparing the test set

We have to do all of those same things for the test set.

**Group Exercise:** Turn this into a function so that you can use it to prepare both the training and test sets.

In [22]:
test_texts = tokenizer.texts_to_sequences(test_data)

# Create the sequences
predictor_sequences_test = []
targets_test = []
for text in test_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences_test.append(curr_predictor_sequence)
        targets_test.append(curr_target)
        
# Pad sequences to ensure uniform length
predictor_sequences_padded_test = pad_sequences(predictor_sequences_test, maxlen=sequence_length, padding='pre')

# Convert target to one-hot encoding
target_word_one_hot_test = to_categorical(targets_test, num_classes=vocabulary_size)

print(predictor_sequences_padded_test)
print(target_word_one_hot_test)

[[ 143  143 3731  445 6880]
 [ 143 3731  445 6880  484]
 [3731  445 6880  484   25]
 ...
 [  33  281   32  894    7]
 [ 281   32  894    7    3]
 [  32  894    7    3  607]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:

def prepare_data(data, seq_len, tokenizer):
    texts = tokenizer.texts_to_sequences(data)
    vocabulary_size = len(tokenizer.word_index) + 1
    
    # Create the sequences
    predictor_sequences = []
    targets = []
    for text in texts:
        for i in range(seq_len, len(text)):
            # Take the sequence of tokens as input and the next token as target
            curr_target = text[i]
            curr_predictor_sequence = text[i-seq_len:i]
            predictor_sequences.append(curr_predictor_sequence)
            targets.append(curr_target)
            
    # Pad sequences to ensure uniform length
    predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=seq_len, padding='pre')

    # Convert target to one-hot encoding
    target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

    return predictor_sequences_padded, target_word_one_hot

In [24]:
predictor_sequences_padded_test, target_word_one_hot_test = prepare_data(test_data, sequence_length, tokenizer)
print(predictor_sequences_padded_test)
print(target_word_one_hot_test)


[[ 143  143 3731  445 6880]
 [ 143 3731  445 6880  484]
 [3731  445 6880  484   25]
 ...
 [  33  281   32  894    7]
 [ 281   32  894    7    3]
 [  32  894    7    3  607]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Designing the Neural Network

We'll start with a simple network like the one we used for text classification.

In [25]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
865/865 [==============================] - 5s 5ms/step - loss: 7.7091 - accuracy: 0.0455 - val_loss: 7.5106 - val_accuracy: 0.0490
Epoch 2/10
865/865 [==============================] - 4s 5ms/step - loss: 6.9437 - accuracy: 0.0631 - val_loss: 7.5948 - val_accuracy: 0.0731
Epoch 3/10
865/865 [==============================] - 4s 5ms/step - loss: 6.4816 - accuracy: 0.0858 - val_loss: 7.8779 - val_accuracy: 0.0849
Epoch 4/10
865/865 [==============================] - 5s 5ms/step - loss: 6.0830 - accuracy: 0.1072 - val_loss: 8.1970 - val_accuracy: 0.0897
Epoch 5/10
865/865 [==============================] - 4s 5ms/step - loss: 5.6764 - accuracy: 0.1341 - val_loss: 8.5932 - val_accuracy: 0.0937
Epoch 6/10
865/865 [==============================] - 4s 5ms/step - loss: 5.2100 - accuracy: 0.1787 - val_loss: 9.1442 - val_accuracy: 0.0927
Epoch 7/10
865/865 [==============================] - 4s 5ms/step - loss: 4.6721 - accuracy: 0.2335 - val_loss: 9.8598 - val_accuracy: 0.0865
Epoch 

## Testing the final model on the test set

In [26]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

214/214 [==============================] - 0s 2ms/step - loss: 13.9154 - accuracy: 0.0747
Test accuracy: 7.47%


## Text Generation

We can use this model to successively generate words based on previous ones - like our Markov sequence on steroids.

Let's see how this works

In [27]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
print(tokens_list)

tokens_array = np.array(tokens_list)
print(tokens_array)

[[1, 91, 20, 12, 19]]
[[ 1 91 20 12 19]]


the model will predict probabilities for each possible word in the output

In [28]:
predicted_probabilities = model.predict(tokens_array,verbose=0)
print(predicted_probabilities)
print("We get a probability for each of the",len(predicted_probabilities[0]),"words")

[[1.4457838e-24 7.3149940e-04 2.6042967e-06 ... 2.2408966e-17
  2.0150673e-16 3.8289548e-27]]
We get a probability for each of the 7867 words


then we could get the word associated with the highest probability

In [29]:
predicted_index = np.argmax(predicted_probabilities)
print("word index:",predicted_index)
predicted_word = tokenizer.index_word[predicted_index]
print("word:",predicted_word)

word index: 18
word: is


or you could generate a random word according the these probabilities (like with did with Markov text generation)

### putting it all together

In [33]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
tokens = tokens_list[0]

for i in range(50):
    curr_seq = tokens[-sequence_length:]
    curr_array = np.array([curr_seq])
    predicted_probabilities = model.predict(curr_array,verbose=0)
    predicted_index = np.argmax(predicted_probabilities)
    predicted_word = tokenizer.index_word[predicted_index]
    print(predicted_word+" ",end="")
    tokens.append(predicted_index)


is selling its tyco global network to help 39 s positive test day 5 million on a quarter earlier old players announced monday that it is selling its tyco global network to help 39 s positive test day 5 million on a quarter earlier old players announced monday that it 

## Applied Exploration

Pick another dataset and get it working with this code
* you will likely need to prepare the text a little differently - do you need to first break it into sentences?
* describe your dataset and what you did to prepare it

Perform a neural language modeling experiment
* experiment with something to try to find a well-performing model
    * sliding window size
    * number of hidden nodes in the network
    * learning rate
* describe what you did and write up an interpretation of your results

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from keras import optimizers

# load the dataset
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

x = np.concatenate((x_train, x_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_sequences(x)

max_sequence_length = 100  # You can set a different maximum sequence length
x_padded = pad_sequences(x, maxlen=max_sequence_length, padding='post', truncating='post')

x_train, x_temp, y_train, y_temp = train_test_split(x_padded, y, train_size = 0.8, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# different learning rates
learning_rates = [0.01, 0.001, 0.0001, 0.00001]  # Add or remove learning rates as needed

for learning_rate in learning_rates:
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    
    # Compile the model with the current learning rate
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])
    
    # Train the model
    history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_val, y_val))
    
    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f'Learning Rate: {learning_rate}')
    print(f'Test Accuracy: {test_acc}')



Epoch 1/5
1250/1250 [==============================] - 53s 42ms/step - loss: 0.5760 - accuracy: 0.6862 - val_loss: 0.4885 - val_accuracy: 0.7774
Epoch 2/5
1250/1250 [==============================] - 51s 41ms/step - loss: 0.4416 - accuracy: 0.7987 - val_loss: 0.4544 - val_accuracy: 0.7938
Epoch 3/5
1250/1250 [==============================] - 51s 41ms/step - loss: 0.3977 - accuracy: 0.8267 - val_loss: 0.4549 - val_accuracy: 0.7980
Epoch 4/5
1250/1250 [==============================] - 50s 40ms/step - loss: 0.3751 - accuracy: 0.8363 - val_loss: 0.4625 - val_accuracy: 0.8004
Epoch 5/5
157/157 [==============================] - 2s 11ms/step - loss: 0.4637 - accuracy: 0.7904


Learning Rate: 0.01
Test Accuracy: 0.7904000282287598


Epoch 1/5
1250/1250 [==============================] - 52s 41ms/step - loss: 0.3194 - accuracy: 0.8668 - val_loss: 0.4692 - val_accuracy: 0.8034
Epoch 2/5
1250/1250 [==============================] - 47s 38ms/step - loss: 0.2801 - accuracy: 0.8876 - val_loss: 0.4801 - val_accuracy: 0.8042
Epoch 3/5
1250/1250 [==============================] - 48s 39ms/step - loss: 0.2513 - accuracy: 0.9018 - val_loss: 0.4926 - val_accuracy: 0.8012
Epoch 4/5
1250/1250 [==============================] - 49s 39ms/step - loss: 0.2280 - accuracy: 0.9124 - val_loss: 0.5114 - val_accuracy: 0.8042
Epoch 5/5
157/157 [==============================] - 2s 11ms/step - loss: 0.5309 - accuracy: 0.7964


Learning Rate: 0.001
Test Accuracy: 0.7964000105857849
Epoch 1/5
1250/1250 [==============================] - 51s 40ms/step - loss: 0.1845 - accuracy: 0.9333 - val_loss: 0.5323 - val_accuracy: 0.7994
Epoch 2/5
1250/1250 [==============================] - 50s 40ms/step - loss: 0.1798 - accuracy: 0.9356 - val_loss: 0.5370 - val_accuracy: 0.7996
Epoch 3/5
1250/1250 [==============================] - 49s 39ms/step - loss: 0.1761 - accuracy: 0.9373 - val_loss: 0.5419 - val_accuracy: 0.8002
Epoch 4/5
1250/1250 [==============================] - 49s 39ms/step - loss: 0.1730 - accuracy: 0.9387 - val_loss: 0.5456 - val_accuracy: 0.7994
Epoch 5/5
157/157 [==============================] - 2s 11ms/step - loss: 0.5469 - accuracy: 0.7992


Learning Rate: 0.0001
Test Accuracy: 0.7991999983787537
Epoch 1/5
1250/1250 [==============================] - 48s 38ms/step - loss: 0.1665 - accuracy: 0.9414 - val_loss: 0.5497 - val_accuracy: 0.7986
Epoch 2/5
1250/1250 [==============================] - 48s 38ms/step - loss: 0.1662 - accuracy: 0.9414 - val_loss: 0.5501 - val_accuracy: 0.7988
Epoch 3/5
1250/1250 [==============================] - 49s 39ms/step - loss: 0.1659 - accuracy: 0.9416 - val_loss: 0.5504 - val_accuracy: 0.7988
Epoch 4/5
1250/1250 [==============================] - 49s 39ms/step - loss: 0.1657 - accuracy: 0.9418 - val_loss: 0.5508 - val_accuracy: 0.7990
Epoch 5/5
157/157 [==============================] - 2s 11ms/step - loss: 0.5486 - accuracy: 0.7976
Learning Rate: 1e-05
Test Accuracy: 0.7975999712944031


<div style="background: #007acc; color: white; border-radius: 10px; padding: 10px; width: fit-content; max-width: 70%; font-size: 1rem;">
    a learning rate around 0.001 appears to strike a balance between training stability and model performance for this sentiment analysis task on the IMDb dataset. Further tuning with other parameters could give better results
</div>

In [19]:
from keras import optimizers

#an example on changing the learning rate
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=["accuracy"])